# How to multi-task in python

In [1]:
import numpy as np
import threading
import time

## 1. Waiting for tasks to complete

Here the python `threading` module is used to enable more explicit control of thread processes. This approach can be useful when waiting on the result of a slow or external process such as an API call or database query. A `slow_calculation` function updating a global variable `result` and `thread.join()` is used to ensure that the slow function completes before progressing. 

In [2]:
result = None

def slow_calculation():
    
    # here goes some long calculation
    rand = np.random.randint(low=2,high=8)
    print(f'Waiting {rand}....')
    time.sleep(rand)
   
    # when the calculation is done, the result is stored in a global variable
    global result
    result = rand**2

def main():
    thread = threading.Thread(target=slow_calculation())
    thread.start()
 
    # dont do this
    # while result is None:
    #     pass
    
    # Do this, wait here for the result to be available before continuing
    thread.join()
   
    print('The result is', result)
    
main()

Waiting 2....
The result is 4


Below is a similar, but this time instead of waiting for the whole slow function to complete we use an event `result_available = threading.Event()` to trigger the continuation of the thread. 

In [3]:
result = None
result_available = threading.Event()

def slow_calculation():
    
    # here goes some long calculation
    rand = np.random.randint(low=2,high=8)
    print(f'Waiting {rand}....')
    time.sleep(rand)
   
    # when the calculation is done, the result is stored in a global variable
    global result
    result = rand**2
    result_available.set()
    
    # do some more work before exiting the thread
    time.sleep(2)
    print('thread finished')

def main():
    thread = threading.Thread(target=slow_calculation())
    thread.start()
 
    # wait here for the result to be available before continuing
    result_available.wait()
   
    print('The result is', result)
    
main()

Waiting 5....
thread finished
The result is 25


## 2. Asynchronous processing

*That first part is the most critical thing to understand - Python code can now basically run in one of two "worlds", either synchronous or asynchronous. You should think of them as relatively separate, having different libraries and calling styles but sharing variables and syntax.*

*In the synchronous world, the Python that's been around for decades, you call functions directly and everything gets processed as it's written on screen. Your only built-in option for running code in parallel in the same process is threads.*

*In the asynchronous world, things change around a bit. Everything runs on a central event loop, which is a bit of core code that lets you run several coroutines at once. Coroutines run synchronously until they hit an await and then they pause, give up control to the event loop, and something else can happen.*

- https://www.aeracode.org/2018/02/19/python-async-simplified/
- https://stackoverflow.com/questions/39952799/python-3-5-async-await-with-real-code-example

In [4]:
import asyncio

In [5]:
async def slow_calculation(max=5):
    rand = np.random.randint(low=2,high=max)
    print(f'Waiting {rand}....')
    await asyncio.sleep(rand)

In [6]:
# slow_calculation()

In [9]:
async def main():
     await asyncio.wait([
                         slow_calculation(i) for i in np.arange(3,7)
                        ])

loop = asyncio.get_event_loop()

In [11]:
loop.run_until_complete(main())

RuntimeError: This event loop is already running

In [14]:
loop.close()

C:\ProgramData\Miniconda3\lib\ast.py:50: RuntimeWarning: coroutine 'main' was never awaited
  return compile(source, filename, mode, flags,


RuntimeError: Cannot close a running event loop